In [22]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
import datetime as dt
from pyspark.sql import functions as F
from pyspark.sql import Window

import os

# Config

In [24]:
# ==========================================
# 사용자 설정 (원하는 지역과 시군구를 입력하세요)
# ==========================================
REGION = "경기도"          # "ex.서울특별시"
SIGUNGU = "용인시 처인구"    # "ex.강남구"

# 베이스 경로
OUTPUT_BASE = "/opt/spark/data/output/silver_stage_1"
ADDRESS_BASE = "/opt/spark/data/address/parquet/road"
COORD_BASE = "opt/spark/data/address/parquet/coord"
BUILDING_BASE   = "/opt/spark/data/buildingLeader/parquet"
TOJI_BASE       = "/opt/spark/data/tojiSoyuJeongbo/parquet"
REST_OWNER_BASE = "/opt/spark/data/restaurant_owner/parquet"


# 데이터 필터용(축약형) 매핑
REGION_MAP = {
    "서울특별시": "서울",
    "부산광역시": "부산",
    "대구광역시": "대구",
    "인천광역시": "인천",
    "광주광역시": "광주",
    "대전광역시": "대전",
    "울산광역시": "울산",
    "세종특별자치시": "세종",
    "경기도": "경기",
    "강원도": "강원",
    "충청북도": "충북",
    "충청남도": "충남",
    "전라북도": "전북",
    "전라남도": "전남",
    "경상북도": "경북",
    "경상남도": "경남",
    "제주특별자치도": "제주",
}

REGION_SHORT = REGION_MAP[REGION]
print("선택된 REGION 축약형: ", REGION_SHORT)

# 데이터 필터용(축약형) 매핑
REGION_ENG_MAP = {
    "서울특별시": "seoul",
    "부산광역시": "busan",
    "대구광역시": "daegu",
    "인천광역시": "incheon",
    "광주광역시": "gwangju",
    "대전광역시": "daejeon",
    "울산광역시": "ulsan",
    "세종특별자치시": "sejong",
    "경기도": "gyunggi",
    "강원도": "gangwon",
    "충청북도": "chungbuk",
    "충청남도": "chungnam",
    "전라북도": "jeonbuk",
    "전라남도": "jeonnam",
    "경상북도": "gyeongbuk",
    "경상남도": "gyeongnam",
    "제주특별자치도": "jeju",
}

REGION_ENG = REGION_ENG_MAP[REGION]
print("선택된 REGION 영어: ", REGION_ENG)

선택된 REGION 축약형:  경기
선택된 REGION 영어:  gyunggi


In [3]:
# Spark Session 설정
spark = SparkSession.builder \
    .appName(f'silver_stage_1_{REGION}_{SIGUNGU.replace(" ", "_")}') \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()

Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/18 14:56:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 원천 데이터 로드 + 지역/시군구 필터링

In [4]:
address_path = f"{ADDRESS_BASE}/rnaddrkor_{REGION_ENG}.parquet"
address_df = spark.read.option("mergeSchema", "false").parquet(address_path)

address_df.printSchema

address_filtered_df = (
    address_df
    .filter(F.col("시도명") == REGION)
    .filter(F.col("시군구명").startswith(SIGUNGU))
    .withColumn(
        "pnu_land_gb",
        F.when(F.col("산여부") == 1, F.lit("2")).otherwise(F.lit("1"))
    )
    .withColumn(
        "PNU코드",
        F.concat(
            F.col("법정동코드"),
            F.col("pnu_land_gb"),
            F.lpad(F.col("지번본번(번지)").cast("string"), 4, "0"),
            F.lpad(F.coalesce(F.col("지번부번(호)"), F.lit(0)).cast("string"), 4, "0")
        )
    )
    .select(
        F.col("PNU코드").cast("string"),

        F.concat_ws(
            " ",
            F.col("시도명"),
            F.col("시군구명"),
            F.col("도로명"),
            F.concat(
                F.col("건물본번").cast("string"),
                F.when(
                    (F.col("건물부번").isNotNull()) & (F.col("건물부번") != 0),
                    F.concat(F.lit("-"), F.col("건물부번").cast("string"))
                ).otherwise(F.lit(""))
            )
        ).alias("도로명주소")
    )
)

address_filtered_df.select(
    F.substring("PNU코드", 1, 5).alias("sigungu_code")
).distinct().show(truncate=False)


SIGUNGU_CODE = address_filtered_df.select(F.substring("PNU코드", 1, 5).alias("sigungu")).first()["sigungu"]

address_filtered_df.printSchema()
print(address_filtered_df.count())
address_filtered_df.show(5, truncate=False)
print("SIGUNGU_CODE =", SIGUNGU_CODE)

+------------+
|sigungu_code|
+------------+
|41220       |
+------------+

root
 |-- PNU코드: string (nullable = true)
 |-- 도로명주소: string (nullable = false)

60761
+-------------------+---------------------------+
|PNU코드            |도로명주소                 |
+-------------------+---------------------------+
|4122010100101590005|경기도 평택시 경기대로 1337|
|4122010100101590105|경기도 평택시 경기대로 1339|
|4122010100101590000|경기도 평택시 경기대로 1341|
|4122010100108120006|경기도 평택시 경기대로 1345|
|4122010100108120005|경기도 평택시 경기대로 1347|
+-------------------+---------------------------+
only showing top 5 rows
SIGUNGU_CODE = 41220


In [5]:
# 날짜 계산 (직전 분기 말일 기준)
today = dt.date.today()
cur_q = (today.month - 1) // 3 + 1
prev_q = cur_q - 1
prev_q_year = today.year
if prev_q == 0:
    prev_q = 4
    prev_q_year -= 1
prev_q_last_month = prev_q * 3

# 데이터 로드 경로
building_path = f"{BUILDING_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"
toji_path     = f"{TOJI_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"
rest_owner_path  = f"{REST_OWNER_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"

In [6]:
# 1) Building
building_df = spark.read.option("mergeSchema", "false").parquet(building_path)
building_gg_df = building_df.filter(F.col("region") == REGION_SHORT)
#if SIGUNGU_CODE:
    #building_gg_df = building_gg_df.filter(F.col("시군구_코드").startswith(SIGUNGU_CODE))

print("건축물대장 스키마:")
building_df.printSchema()


# 2) Toji
toji_df = spark.read.option("mergeSchema", "false").parquet(toji_path)
toji_gg_df = toji_df.filter(F.col("region") == REGION_SHORT)
if SIGUNGU_CODE:
    toji_gg_df = toji_gg_df.filter(F.col("법정동코드").startswith(SIGUNGU_CODE))

print("토지소유정보 스키마:")
toji_df.printSchema()

print("Distinct 지목 리스트:")
toji_df.select("지목").distinct().orderBy("지목").show(1000, truncate=False)

print("식품안전나라 스키마:")
# 3) Owner
owner_df = spark.read.option("mergeSchema", "false").parquet(rest_owner_path)
owner_gg_df = owner_df.filter((F.col("region") == REGION_SHORT) & F.col("소재지").startswith(f'{REGION} {SIGUNGU}'))

owner_df.printSchema()

건축물대장 스키마:
root
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 대장_구분_코드: string (nullable = true)
 |-- 대장_구분_코드_명: string (nullable = true)
 |-- 대장_종류_코드: string (nullable = true)
 |-- 대장_종류_코드_명: string (nullable = true)
 |-- 대지_위치: string (nullable = true)
 |-- 도로명_대지_위치: string (nullable = true)
 |-- 건물_명: string (nullable = true)
 |-- 시군구_코드: string (nullable = true)
 |-- 법정동_코드: string (nullable = true)
 |-- 대지_구분_코드: string (nullable = true)
 |-- 번: string (nullable = true)
 |-- 지: string (nullable = true)
 |-- 특수지_명: string (nullable = true)
 |-- 블록: string (nullable = true)
 |-- 로트: string (nullable = true)
 |-- 외필지_수: string (nullable = true)
 |-- 새주소_도로_코드: string (nullable = true)
 |-- 새주소_법정동_코드: string (nullable = true)
 |-- 새주소_지상지하_코드: string (nullable = true)
 |-- 새주소_본_번: string (nullable = true)
 |-- 새주소_부_번: string (nullable = true)
 |-- 동_명: string (nullable = true)
 |-- 주_부속_구분_코드: string (nullable = true)
 |-- 주_부속_구분_코드_명: string (nullable = true)
 |-- 대지_면적(㎡):

[Stage 13:=========================================>               (8 + 3) / 11]

+----------+
|지목      |
+----------+
|공원      |
|공장용지  |
|과수원    |
|광천지    |
|구거      |
|답        |
|대        |
|도로      |
|목장용지  |
|묘지      |
|사적지    |
|수도용지  |
|양어장    |
|염전      |
|유원지    |
|유지      |
|임야      |
|잡종지    |
|전        |
|제방      |
|종교용지  |
|주유소용지|
|주차장    |
|창고용지  |
|철도용지  |
|체육용지  |
|하천      |
|학교용지  |
+----------+

식품안전나라 스키마:
root
 |-- 번호: long (nullable = true)
 |-- 인허가번호: long (nullable = true)
 |-- 업체명: string (nullable = true)
 |-- 업종: string (nullable = true)
 |-- 대표자: string (nullable = true)
 |-- 소재지: string (nullable = true)
 |-- 인허가기관: string (nullable = true)
 |-- 영업상태: string (nullable = true)
 |-- 비고: double (nullable = true)
 |-- region: string (nullable = true)



# Building Pruning (PNU 고유번호 생성 + 필요한 컬럼만 select)

In [7]:
building_gg_df = building_gg_df.withColumn(
    "고유번호",
    F.concat(
        F.col("시군구_코드"),        # 5
        F.col("법정동_코드"),        # 5
        F.when(F.col("대지_구분_코드") == "0", F.lit("1"))  # 대지 -> 1
         .when(F.col("대지_구분_코드") == "1", F.lit("2"))  # 산   -> 2
         .otherwise(F.col("대지_구분_코드")),
        F.lpad(F.col("번"), 4, "0"), # 본번
        F.lpad(F.col("지"), 4, "0")  # 부번
    )
)

building_gg_df = (
    building_gg_df
    .select(
        F.col("관리_건축물대장_PK"),
        F.col("고유번호"),
        F.col("옥외_자주식_면적(㎡)").alias("옥외자주식면적"),
        F.col("대장_구분_코드")
    )
)

building_gg_df.printSchema()
print(f"전체 건축물대장 row 수: {building_gg_df.count()}")
building_gg_df.show(5, truncate=False)

root
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 고유번호: string (nullable = true)
 |-- 옥외자주식면적: string (nullable = true)
 |-- 대장_구분_코드: string (nullable = true)

전체 건축물대장 row 수: 1324795
+------------------+-------------------+--------------+--------------+
|관리_건축물대장_PK|고유번호           |옥외자주식면적|대장_구분_코드|
+------------------+-------------------+--------------+--------------+
|1099110746        |4122010100108840002|0             |2             |
|109918059         |4122025930103930014|0             |1             |
|1099111239        |4122010300104390000|0             |1             |
|10991100508032    |4122037029100160002|27.5          |1             |
|1099110636        |4122010100201370000|0             |1             |
+------------------+-------------------+--------------+--------------+
only showing top 5 rows


# Toji Pruning (개인 소유, 가장 최근 소유권 변동 정보만 필터링)

In [8]:
all_count = toji_gg_df.count()
print(f"전체 토지 수: {all_count}")

toji_gg_df = (
    toji_gg_df
    # 1) 조건 필터
    #.filter(F.col("공유인수") == 0)
    .filter(F.col("소유구분코드") == "01")
    # 2) 필요한 컬럼만 선택
    .select(
        F.col("고유번호").cast("string"),
        F.col("법정동명"),
        F.col("지번"),
        F.col("소유권변동일자"),
        F.col("토지면적"),
        F.col("지목"),
    )

    # 3) 날짜 파싱
    .withColumn(
        "소유권변동일자_dt",
        F.to_date(F.col("소유권변동일자"), "yyyy-MM-dd")
    )

    # 4) 고유번호별 최신 1행
    .withColumn(
        "rn",
        F.row_number().over(
            Window.partitionBy("고유번호")
                  .orderBy(F.col("소유권변동일자_dt").desc_nulls_last())
        )
    )
    .filter(F.col("rn") == 1)
    .drop("rn", "소유권변동일자_dt")

    # 5) 지번 분리
    .withColumn(
        "본번",
        F.split(F.col("지번"), "-").getItem(0)
    )
    .withColumn(
        "부번",
        F.when(
            F.size(F.split(F.col("지번"), "-")) > 1,
            F.split(F.col("지번"), "-").getItem(1)
        ).otherwise(F.lit(None))
    )
)

# 확인
toji_gg_df.printSchema()
print(f"필터링 후 토지 수: {toji_gg_df.count()} ({toji_gg_df.count() / all_count * 100:.2f}%)")
toji_gg_df.show(10, truncate=False)

전체 토지 수: 944829
root
 |-- 고유번호: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)



필터링 후 토지 수: 198560 (21.02%)


[Stage 36:===========================================>              (3 + 1) / 4]

+-------------------+--------------------+-----+--------------+--------+----+----+----+
|고유번호           |법정동명            |지번 |소유권변동일자|토지면적|지목|본번|부번|
+-------------------+--------------------+-----+--------------+--------+----+----+----+
|4122010100100020001|경기도 평택시 서정동|2-1  |2014-02-13    |194.0   |대  |2   |1   |
|4122010100100020006|경기도 평택시 서정동|2-6  |2023-05-10    |35.36   |대  |2   |6   |
|4122010100100020008|경기도 평택시 서정동|2-8  |2022-05-23    |46.3    |대  |2   |8   |
|4122010100100020026|경기도 평택시 서정동|2-26 |2013-09-02    |23.74   |대  |2   |26  |
|4122010100100020079|경기도 평택시 서정동|2-79 |2021-12-17    |331.0   |임야|2   |79  |
|4122010100100020084|경기도 평택시 서정동|2-84 |2018-10-31    |331.0   |임야|2   |84  |
|4122010100100020086|경기도 평택시 서정동|2-86 |2024-11-27    |329.0   |대  |2   |86  |
|4122010100100200001|경기도 평택시 서정동|20-1 |2025-03-21    |24.58   |대  |20  |1   |
|4122010100100200040|경기도 평택시 서정동|20-40|2023-07-19    |13.93   |대  |20  |40  |
|4122010100100200041|경기도 평택시 서정동|20-41|2023-07-19    |15.27   |

# Restaurant Pruning (필요한 컬럼만 + 중복 제거)

In [9]:
# =========================
# Restaurant pruning
# =========================
# =========================
# Restaurant pruning
# =========================
owner_gg_df_clean = (
    owner_gg_df

    # -----------------------------
    # 1. 괄호 제거
    # -----------------------------
    .withColumn(
        "소재지_tmp",
        F.regexp_replace(F.col("소재지"), r"\s*\([^)]*\)", "")
    )

    # -----------------------------
    # 2. 쉼표 뒤 제거
    # -----------------------------
    .withColumn(
        "소재지_tmp",
        F.regexp_replace(F.col("소재지_tmp"), r",.*$", "")
    )

    # -----------------------------
    # 3. 도로명 앞 행정동 제거
    # (고림동 고림로 123 → 고림로 123)
    # -----------------------------
    .withColumn(
        "소재지_tmp",
        F.regexp_replace(
            F.col("소재지_tmp"),
            r'\s+\S+(동|읍|면|리)\s+(?=\S+(로|길))',
            ' '
        )
    )

    # -----------------------------
    # 4. 공백 정리
    # -----------------------------
    .withColumn(
        "소재지_정제",
        F.trim(F.regexp_replace(F.col("소재지_tmp"), r'\s+', ' '))
    )

    .select(
        F.col("업체명"),
        F.col("대표자"),
        F.col("소재지_정제"),
    )
    .filter(F.col("업체명").isNotNull())
)

# =========================
# Owner 중복 제거
# =========================
all_count = owner_gg_df_clean.count()

owner_gg_df_clean = owner_gg_df_clean.dropDuplicates(["업체명", "소재지_정제"])

print(f"owner (업체명, 소재지) 중복 제거 후: {owner_gg_df_clean.count()} ({owner_gg_df_clean.count() / all_count * 100:.2f}%)")

owner_gg_df_clean.show(10, truncate=False)

owner (업체명, 소재지) 중복 제거 후: 11026 (89.24%)
+-----------------------------------+-------------------------+------------------------------------------------------------------------------+
|업체명                             |대표자                   |소재지_정제                                                                   |
+-----------------------------------+-------------------------+------------------------------------------------------------------------------+
|(DNO)디엔오                        |오*석                    |경기도 평택시 세교6로 12-47                                                   |
|(The)착한                          |김*선                    |경기도 평택시 서동대로 2043                                                   |
|(돈야우야)모산골                   |한*숙                    |경기도 평택시 동삭1로22번길 19-6                                              |
|(신)열날개                         |성*영                    |경기도 평택시 함박산8길 7                                                     |
|(유)아웃백스테이크하우스 평택역사점|S*NG DAVID H

# Restaurant + Address Left Join (도로명주소 -> 법정동코드 컬럼 추가)

In [10]:
o = owner_gg_df_clean.alias("o")
a = address_filtered_df.alias("a")

restaurant_address_df = (
    o.join(
        a,
        F.col("o.소재지_정제") == F.col("a.도로명주소"),
        how="left"
    )
    .filter(F.col("도로명주소").isNotNull())
    .drop("도로명주소")
)

restaurant_address_df.printSchema()
print(f"owner 도로명주소 매칭 후: {restaurant_address_df.count()} ({restaurant_address_df.count() / owner_gg_df_clean.count() * 100:.2f}%)")
restaurant_address_df.show(10, truncate=False)

o = owner_gg_df_clean.alias("o")
a = address_filtered_df.alias("a")

unmatched_df = (
    o.join(
        a,
        F.col("o.소재지_정제") == F.col("a.도로명주소"),
        how="left"
    )
    .filter(F.col("a.도로명주소").isNull())   # ← 매칭 실패만
)

print("매칭 실패 개수:", unmatched_df.count())
unmatched_df.show(20, truncate=False)


root
 |-- 업체명: string (nullable = true)
 |-- 대표자: string (nullable = true)
 |-- 소재지_정제: string (nullable = true)
 |-- PNU코드: string (nullable = true)

owner 도로명주소 매칭 후: 10912 (98.97%)
+-----------------------------------+-------------------------+--------------------------------+-------------------+
|업체명                             |대표자                   |소재지_정제                     |PNU코드            |
+-----------------------------------+-------------------------+--------------------------------+-------------------+
|(DNO)디엔오                        |오*석                    |경기도 평택시 세교6로 12-47     |4122012000105840010|
|(The)착한                          |김*선                    |경기도 평택시 서동대로 2043     |4122034026100010002|
|(돈야우야)모산골                   |한*숙                    |경기도 평택시 동삭1로22번길 19-6|4122011900100000000|
|(신)열날개                         |성*영                    |경기도 평택시 함박산8길 7       |4122012800126580004|
|(유)아웃백스테이크하우스 평택역사점|S*NG DAVID HOSUP (송호섭)|경기도 평택시 평택로 51         |412201

# 토지 필터링
1. 토지에 건물이 없거나
2. 건물이 1개 있고, 일반 건물이며, 음식점이 있는 토지만 필터링

In [11]:
t = toji_gg_df.alias("t")
b = building_gg_df.alias("b")

toji_building_gg_df = (
    t.join(b, F.col("t.고유번호") == F.col("b.고유번호"), "left")
     .drop(F.col("b.고유번호"))
)

toji_building_gg_df.printSchema()
print(toji_building_gg_df.count())
toji_building_gg_df.show(5, truncate=False)

root
 |-- 고유번호: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 옥외자주식면적: string (nullable = true)
 |-- 대장_구분_코드: string (nullable = true)



210096


[Stage 101:============>    (3 + 1) / 4][Stage 102:======>          (2 + 3) / 5]

+-------------------+--------------------+----+--------------+--------+----+----+----+------------------+--------------+--------------+
|고유번호           |법정동명            |지번|소유권변동일자|토지면적|지목|본번|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|
+-------------------+--------------------+----+--------------+--------+----+----+----+------------------+--------------+--------------+
|4122010100100020001|경기도 평택시 서정동|2-1 |2014-02-13    |194.0   |대  |2   |1   |10991100179000    |0             |1             |
|4122010100100020006|경기도 평택시 서정동|2-6 |2023-05-10    |35.36   |대  |2   |6   |1099125856        |81.5          |2             |
|4122010100100020008|경기도 평택시 서정동|2-8 |2022-05-23    |46.3    |대  |2   |8   |1099152176        |80.5          |2             |
|4122010100100020026|경기도 평택시 서정동|2-26|2013-09-02    |23.74   |대  |2   |26  |10991100244732    |24            |2             |
|4122010100100020026|경기도 평택시 서정동|2-26|2013-09-02    |23.74   |대  |2   |26  |10991100244718    |24            |2             |
+--------

### 건물이 1개 이하인 토지 필터링

In [12]:
pk_cnt_df = (
    toji_building_gg_df
    .groupBy("고유번호")
    .agg(F.count("관리_건축물대장_PK").alias("pk_cnt"))
)

toji_binary_building_gg_df = (
    toji_building_gg_df
    .join(
        pk_cnt_df.filter(F.col("pk_cnt") <= 1),
        on="고유번호",
        how="inner"
    )
    .drop("pk_cnt")
)

print(toji_binary_building_gg_df.count())
toji_binary_building_gg_df.show(5, truncate=False)

192874


[Stage 121:============================>                            (2 + 2) / 4]

+-------------------+--------------------+----+--------------+--------+----+----+----+------------------+--------------+--------------+
|고유번호           |법정동명            |지번|소유권변동일자|토지면적|지목|본번|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|
+-------------------+--------------------+----+--------------+--------+----+----+----+------------------+--------------+--------------+
|4122010100100020001|경기도 평택시 서정동|2-1 |2014-02-13    |194.0   |대  |2   |1   |10991100179000    |0             |1             |
|4122010100100020006|경기도 평택시 서정동|2-6 |2023-05-10    |35.36   |대  |2   |6   |1099125856        |81.5          |2             |
|4122010100100020008|경기도 평택시 서정동|2-8 |2022-05-23    |46.3    |대  |2   |8   |1099152176        |80.5          |2             |
|4122010100100020079|경기도 평택시 서정동|2-79|2021-12-17    |331.0   |임야|2   |79  |NULL              |NULL          |NULL          |
|4122010100100020084|경기도 평택시 서정동|2-84|2018-10-31    |331.0   |임야|2   |84  |NULL              |NULL          |NULL          |
+----------

In [13]:
toji_binary_building_gg_df.withColumn(
    "건물개수",
    F.when(F.col("관리_건축물대장_PK").isNull(), F.lit(0))
     .otherwise(F.lit(1))
).groupBy("건물개수").count().orderBy("건물개수").show()

[Stage 131:>                                                        (0 + 4) / 4]

+--------+------+
|건물개수| count|
+--------+------+
|       0|151328|
|       1| 41546|
+--------+------+



### 건물이 1개 있고, 일반 건물이며, 음식점이 포함된 토지 필터링

In [14]:
# 건물이 1개 있는 토지만 필터링
toji_with_1_building_gg_df = (
    toji_binary_building_gg_df
    .filter(F.col("관리_건축물대장_PK").isNotNull())
)

print("건물 1개인 토지 수:", toji_with_1_building_gg_df.count())

26/02/18 14:56:35 ERROR TaskSchedulerImpl: Lost executor 1 on 172.22.0.3: Command exited with code 137
26/02/18 14:56:35 WARN TaskSetManager: Lost task 2.0 in stage 138.0 (TID 220) (172.22.0.3 executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/18 14:56:35 WARN TaskSetManager: Lost task 3.0 in stage 138.0 (TID 221) (172.22.0.3 executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/18 14:56:35 WARN TaskSetManager: Lost task 0.0 in stage 140.0 (TID 225) (172.22.0.4 executor 0): FetchFailed(null, shuffleId=42, mapIndex=-1, mapId=-1, reduceId=0, message=
org.apache.spark.shuffle.MetadataFetchFailedException: Missing an output location for shuffle 42 partition 0
	at org.apache.spark.MapOutputTracker$.validateStatus(MapOutputTracker.scala:1781)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11(MapOutputTracker.scala:172

건물 1개인 토지 수: 41546


In [15]:
# 건물 있는데 음식점 아닌거 거르려고 만드는 join table

t = toji_with_1_building_gg_df.alias("t")
r = restaurant_address_df.alias("r")

toji_building_restaurant_gg_df = (
    t.join(
        r,
        F.col("t.고유번호") == F.col("r.PNU코드"),
        how="left"
    )
    .drop(F.col("r.PNU코드"))
    .filter(F.col("업체명").isNotNull())
)

print("건물이 1개이고, 음식점이 있는 토지:", toji_building_restaurant_gg_df.count())
toji_building_restaurant_gg_df.show(5)

건물이 1개이고, 음식점이 있는 토지: 7148


+-------------------+--------------------+-----+--------------+--------+----+----+----+------------------+--------------+--------------+--------------+------------+--------------------------------+
|           고유번호|            법정동명| 지번|소유권변동일자|토지면적|지목|본번|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|        업체명|      대표자|                     소재지_정제|
+-------------------+--------------------+-----+--------------+--------+----+----+----+------------------+--------------+--------------+--------------+------------+--------------------------------+
|4122010100100020001|경기도 평택시 서정동|  2-1|    2014-02-13|   194.0|  대|   2|   1|    10991100179000|             0|             1|   송탄 왕오리|       안*덕| 경기도 평택시 경기대로1376번...|
|4122010100100510015|경기도 평택시 서정동|51-15|    2022-03-30|    6.77|  대|  51|  15|    10991100389571|             0|             2|      포레스트|       송*희|  경기도 평택시 송탄2로19번길 44|
|4122010100100760021|경기도 평택시 서정동|76-21|    2017-08-11|    88.0|  대|  76|  21|        1099141241|             0|            

### 건물 0개 + 건물 1개 (일반건물, 음식점있음) 토지 concat

In [16]:
# 아무 건물도 없는 필지
toji_with_0_building_gg_df = (
    toji_binary_building_gg_df
    .filter(F.col("관리_건축물대장_PK").isNull())
)

# 위에서 만든 join table과 concat을 위해 column 추가
toji_with_0_building_gg_df = (
    toji_with_0_building_gg_df
    .withColumn("업체명", F.lit(None).cast("string"))
    .withColumn("대표자", F.lit(None).cast("string"))
    .withColumn("소재지_정제", F.lit(None).cast("string"))
)

final_toji_df = (
    toji_building_restaurant_gg_df
    .unionByName(toji_with_0_building_gg_df)
)

In [17]:
# 전체 row 수 = (건물1+음식점) + (건물0)
print("final:", final_toji_df.count())
print("건물1+음식점:", toji_building_restaurant_gg_df.count())
print("건물0:", toji_with_0_building_gg_df.count())

# 건물 개수 분포
final_toji_df.withColumn(
    "건물개수",
    F.when(F.col("관리_건축물대장_PK").isNull(), 0).otherwise(1)
).groupBy("건물개수").count().orderBy("건물개수").show()

final_toji_df.show(5)

final: 158476


건물1+음식점: 7148


건물0: 151328


+--------+------+
|건물개수| count|
+--------+------+
|       0|151328|
|       1|  7148|
+--------+------+



[Stage 272:>                                                        (0 + 1) / 1]

+-------------------+--------------------+-----+--------------+--------+----+----+----+------------------+--------------+--------------+---------------------------------+------+-------------------------------+
|           고유번호|            법정동명| 지번|소유권변동일자|토지면적|지목|본번|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|                           업체명|대표자|                    소재지_정제|
+-------------------+--------------------+-----+--------------+--------+----+----+----+------------------+--------------+--------------+---------------------------------+------+-------------------------------+
|4122010100100010000|경기도 평택시 서정동|    1|    2020-05-22|   125.0|  대|   1|NULL|    10991100207237|             0|             1|                       서정동책방| 김*숙|경기도 평택시 경기대로1376번...|
|4122010100100020001|경기도 평택시 서정동|  2-1|    2014-02-13|   194.0|  대|   2|   1|    10991100179000|             0|             1|                      송탄 왕오리| 안*덕|경기도 평택시 경기대로1376번...|
|4122010100100480000|경기도 평택시 서정동|   48|    2025-06-30|   12.85| 

# 같은 본번을 가진 토지 그룹핑 -> 음식점이 있는 그룹만 필터링

In [18]:
group_has_restaurant_df = (
    final_toji_df
    .groupBy("법정동명", "본번")
    .agg(
        F.max(
            F.when(F.col("업체명").isNotNull(), 1).otherwise(0)
        ).alias("has_restaurant")
    )
    .filter(F.col("has_restaurant") == 1)
    .select("법정동명", "본번")
)

filtered_final_toji_df = (
    final_toji_df
    .join(group_has_restaurant_df, on=["법정동명", "본번"], how="inner")
)

print("필터 후 row 수:", filtered_final_toji_df.count())
filtered_final_toji_df.show(5, truncate=False)


26/02/18 14:57:22 ERROR TaskSchedulerImpl: Lost executor 0 on 172.22.0.4: Command exited with code 137
26/02/18 14:57:22 WARN TaskSetManager: Lost task 2.0 in stage 294.0 (TID 475) (172.22.0.4 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/18 14:57:22 WARN TaskSetManager: Lost task 3.0 in stage 294.0 (TID 476) (172.22.0.4 executor 0): ExecutorLostFailure (executor 0 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/18 14:57:22 WARN TaskSetManager: Lost task 3.1 in stage 294.0 (TID 477) (172.22.0.3 executor 2): FetchFailed(BlockManagerId(0, 172.22.0.4, 38609, None), shuffleId=90, mapIndex=1, mapId=454, reduceId=199, message=
org.apache.spark.shuffle.FetchFailedException
	at org.apache.spark.errors.SparkCoreErrors$.fetchFailedError(SparkCoreErrors.scala:426)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.sc

필터 후 row 수: 19352


26/02/18 14:57:40 ERROR TaskSchedulerImpl: Lost executor 2 on 172.22.0.3: Command exited with code 137
26/02/18 14:57:40 WARN TaskSetManager: Lost task 0.0 in stage 338.0 (TID 550) (172.22.0.3 executor 2): ExecutorLostFailure (executor 2 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/18 14:57:40 WARN TaskSetManager: Lost task 1.0 in stage 338.0 (TID 551) (172.22.0.3 executor 2): ExecutorLostFailure (executor 2 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/18 14:57:41 WARN TaskSetManager: Lost task 2.1 in stage 334.0 (TID 553) (172.22.0.4 executor 3): FetchFailed(BlockManagerId(2, 172.22.0.3, 38721, None), shuffleId=101, mapIndex=0, mapId=521, reduceId=0, message=
org.apache.spark.shuffle.FetchFailedException
	at org.apache.spark.errors.SparkCoreErrors$.fetchFailedError(SparkCoreErrors.scala:426)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.sca

+--------------------+----+-------------------+------+--------------+--------+----+----+------------------+--------------+--------------+-----------------------+-------------+------------------------------+
|법정동명            |본번|고유번호           |지번  |소유권변동일자|토지면적|지목|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|업체명                 |대표자       |소재지_정제                   |
+--------------------+----+-------------------+------+--------------+--------+----+----+------------------+--------------+--------------+-----------------------+-------------+------------------------------+
|경기도 평택시 합정동|964 |4122011700109640007|964-7 |2025-08-26    |257.1   |대  |7   |10991100300950    |0             |1             |투다리 배미점          |최*아        |경기도 평택시 중앙로 264      |
|경기도 평택시 합정동|964 |4122011700109640007|964-7 |2025-08-26    |257.1   |대  |7   |10991100300950    |0             |1             |청기와24시감자탕 평택점|Z*ANG CHUNMEI|경기도 평택시 중앙로 264      |
|경기도 평택시 합정동|964 |4122011700109640012|964-12|2014-11-18    |321.4   |대  |12  |1

# 같은 지주 소유로 추정되는 토지 그룹핑 -> 토지대장 발급 리스트
* 같은 본번으로 묶이는 토지 내에서 소유권변동일자가 같으면 같은 사람이 매입한 것으로 추정

In [19]:
toji_group_df = (
    filtered_final_toji_df
    .groupBy("법정동명", "본번", "소유권변동일자")
    .agg(
        F.min("부번").alias("부번_리스트")
    )
    .distinct()
)

print(toji_group_df.count())
toji_group_df.show(10, truncate=False)

12858


26/02/18 14:58:58 ERROR TaskSchedulerImpl: Lost executor 3 on 172.22.0.4: Command exited with code 137
26/02/18 14:58:58 WARN TaskSetManager: Lost task 0.0 in stage 392.0 (TID 634) (172.22.0.4 executor 3): ExecutorLostFailure (executor 3 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/18 14:58:58 WARN TaskSetManager: Lost task 2.0 in stage 392.0 (TID 636) (172.22.0.4 executor 3): ExecutorLostFailure (executor 3 exited caused by one of the running tasks) Reason: Command exited with code 137
                                                                                

+--------------------+----+--------------+-----------+
|법정동명            |본번|소유권변동일자|부번_리스트|
+--------------------+----+--------------+-----------+
|경기도 평택시 가재동|106 |2022-12-27    |1          |
|경기도 평택시 가재동|106 |2023-01-02    |2          |
|경기도 평택시 가재동|143 |2002-09-10    |3          |
|경기도 평택시 가재동|143 |2017-08-22    |5          |
|경기도 평택시 가재동|308 |2018-11-16    |1          |
|경기도 평택시 가재동|308 |2019-05-28    |11         |
|경기도 평택시 가재동|308 |2022-01-21    |12         |
|경기도 평택시 가재동|314 |2010-01-11    |2          |
|경기도 평택시 가재동|82  |2018-10-24    |6          |
|경기도 평택시 가재동|82  |2023-11-14    |4          |
+--------------------+----+--------------+-----------+
only showing top 10 rows


In [20]:
# =========================
# 저장 경로 생성
# =========================
base_output_path = os.path.join(
    OUTPUT_BASE,
    f"{REGION}_{SIGUNGU.replace(' ', '_')}"
)

filtered_path = base_output_path + "_filtered_final_toji"
group_path = base_output_path + "_toji_group"

print("filtered 저장 경로:", filtered_path)
print("group 저장 경로:", group_path)

# =========================
# parquet 저장
# =========================
filtered_final_toji_df.write \
    .mode("overwrite") \
    .parquet(filtered_path)

toji_group_df.write \
    .mode("overwrite") \
    .parquet(group_path)

print("✅ 저장 완료")


filtered 저장 경로: /opt/spark/data/output/silver_stage_1/경기도_평택시_filtered_final_toji
group 저장 경로: /opt/spark/data/output/silver_stage_1/경기도_평택시_toji_group


26/02/18 15:00:06 ERROR TaskSchedulerImpl: Lost executor 4 on 172.22.0.3: Command exited with code 137
26/02/18 15:00:06 WARN TaskSetManager: Lost task 4.0 in stage 429.0 (TID 690) (172.22.0.3 executor 4): ExecutorLostFailure (executor 4 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/18 15:00:24 ERROR TaskSchedulerImpl: Lost executor 5 on 172.22.0.4: Command exited with code 137
26/02/18 15:00:24 WARN TaskSetManager: Lost task 1.0 in stage 469.0 (TID 752) (172.22.0.4 executor 5): ExecutorLostFailure (executor 5 exited caused by one of the running tasks) Reason: Command exited with code 137
26/02/18 15:00:24 WARN TaskSetManager: Lost task 2.0 in stage 469.0 (TID 753) (172.22.0.4 executor 5): ExecutorLostFailure (executor 5 exited caused by one of the running tasks) Reason: Command exited with code 137
                                                                                

✅ 저장 완료


In [21]:
spark.stop()